# Laboratory 2.2.1

Welcome to Lab 2.2.1, which is a continuation of Lab 2.2 on Colab. In this lab, we will guide you on how to apply the model you just trained to detect objects in videos or via your laptop's camera.


## Instructions

Below is a detailed guide to help you understand how to apply YOLO to videos.


### Libraries

In [21]:
import numpy as np
import torch
import cv2
import math
import time
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

### Check GPU

In [ ]:
# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available!")
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("No GPU available, using CPU.")


### Loading the Detection Model

Solution to exercise 1

In [ ]:
# force reload: avoid parameter conflicts when loading new models
model = torch.hub.load('ultralytics/yolov5', 'custom', path= r"[...]\best.pt", force_reload = True)

### Testing with Images



In [ ]:
img_path = [...]

In [ ]:
img = cv2.imread(img_path)

cv2.imshow("test", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
test_detect = model(img_path)
test_detect.show()

### Work with Video

Solution to exercise 2

In [ ]:
input_video_path = [...]  # Path to input video
output_video_path = [...]  # Path to output video

# Open input video
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video parameters
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create video writer to save output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frames from BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Predict with YOLOv5
    results = model(rgb_frame)

    # Get the prediction result as a DataFrame
    df = results.pandas().xyxy[0]

	# Draw prediction boxes on the canvas
    for _, row in df.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        label = row['name']
        score = row['confidence']
        
        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # Draw class labels and confidence levels
        cv2.putText(frame, f'{label} {score:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Record frames to output video
    out.write(frame)

# Free up resources
cap.release()
out.release()
cv2.destroyAllWindows()

